In [ ]:
import pandas as pd
import openai
import json
from collections import defaultdict
import datetime
import io 
import faiss
import numpy as np

from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")


client = openai.OpenAI(api_key=api_key)
LLM_MODEL = "gpt-5-2025-08-07"

# 파일 경로 설정
raw_path = "/home/hyunji/WEO_Chat/weo_test.xlsx"
structured_csv_path = "structured_from_llm.csv"
hierarchical_json_path = "extracted_hierarchical.json"
response_path = "response.txt"

In [ ]:
def excel_to_raw_text(path):
    try:
        xls = pd.ExcelFile(path)
        all_sheets_text = ""
        for sheet_name in xls.sheet_names:
            df = xls.parse(sheet_name, header=None)
            all_sheets_text += f"--- START OF SHEET: {sheet_name} ---\n"
            all_sheets_text += df.to_csv(index=False, header=False)
            all_sheets_text += f"--- END OF SHEET: {sheet_name} ---\n\n"
        return all_sheets_text
    except Exception as e:
        print(f"Error while parsing: {e}")
        return None

restructure_prompt = """
You are a data wrangling expert. You will be given text extracted from a complex, multi-sheet Excel file.
Your task is to understand the pivoted and messy structure and convert it into a clean, normalized CSV format.

Follow these rules precisely:
1.  Identify the main data table in each sheet. Ignore all metadata, titles, comments like 'Back to contents page', and empty rows.
2.  The columns of the output CSV must be: `Product`, `Region`, `Year`, `Scenario`, `Value`.
3.  Unpivot the data. The original file has years and scenarios as columns. You must transform them into rows.
4.  The 'Product' can be determined from the sheet name (e.g., 'Sheet1' for 'Renewables', 'Sheet2' for 'Total') or from context within the sheet.
5.  The 'Scenario' is indicated in the column headers (e.g., 'Stated Policies Scenario', 'Announced Pledges Scenario').
6.  The 'Year' is also in the column headers. Match the correct year and scenario to each value.
7.  The 'Region' is in one of the first few columns of the data rows.
8.  Output ONLY the CSV data, including a header row. Do not include any explanations, comments, or markdown formatting.
"""

print("1단계: LLM으로 비정형 엑셀 데이터 정제 시작...")
raw_excel_text = excel_to_raw_text(raw_path)

if raw_excel_text:
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": restructure_prompt},
            {"role": "user", "content": raw_excel_text}
        ]
    )
    structured_csv_str = response.choices[0].message.content.strip()

    if structured_csv_str.startswith("```csv"):
        structured_csv_str = structured_csv_str[len("```csv"):].strip()
    if structured_csv_str.endswith("```"):
        structured_csv_str = structured_csv_str[:-len("```")].strip()

    with open(structured_csv_path, "w", encoding="utf-8") as f:
        f.write(structured_csv_str)
    print(f"LLM이 정제된 CSV 생성 완료! 파일 저장: {structured_csv_path}")
else:
    print("엑셀 파일 읽기 실패.")
    exit()


In [ ]:
# %%
# --- 3. 2단계: 규칙 기반으로 정형 CSV를 계층적 JSON으로 변환 ---

def create_hierarchical_json_from_csv(csv_data_string):
    try:
        df = pd.read_csv(io.StringIO(csv_data_string))
    except Exception as e:
        print(f"CSV 데이터를 읽는 중 오류 발생: {e}")
        return None

    structured_data = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))

    for _, row in df.iterrows():
        try:
            product = row['Product']
            region = row['Region']
            year = int(row['Year'])
            scenario = row['Scenario']
            value = float(row['Value'])
            
            structured_data[region][year][scenario][product] = value
        except (KeyError, ValueError, TypeError) as e:
            print(f"행 처리 중 오류 발생 (건너뜀): {row}, 오류: {e}")
            continue
            
    return json.loads(json.dumps(structured_data))

print("\n2단계: 규칙 기반으로 CSV를 계층적 JSON으로 변환 시작...")
hierarchical_data = create_hierarchical_json_from_csv(structured_csv_str)

if hierarchical_data:
    with open(hierarchical_json_path, "w", encoding="utf-8") as json_file:
        json.dump(hierarchical_data, json_file, indent=4, ensure_ascii=False)
    print(f"계층적 JSON 변환 완료! 파일 저장: {hierarchical_json_path}")
else:
    print("JSON 변환 실패.")
    exit()

----

In [ ]:
SYSTEM_PROMPT = """
You are a meticulous and insightful data analyst specializing in global energy trends.
Your task is to answer the user's question based exclusively on the provided JSON data context.

Follow these instructions carefully to structure your response:

1.  **Analyze the User's Query:** Understand if the user is asking for a specific fact, a comparison, a calculation, or an inference based on the data.

2.  **Ground Your Answer in Data:** All parts of your response must be directly supported by the provided JSON data. Do not use any external knowledge.

3.  **Perform Necessary Operations:**
    -   For direct questions, find the exact value.
    -   For comparison questions, retrieve all relevant values and compare them.
    -   For calculation questions (e.g., percentage, difference), state the formula you used and the values you plugged in.

4.  **Handle Inferences Carefully:**
    -   If the user asks for an opinion or inference (e.g., "potential risks"), you are allowed to make a logical deduction based on the patterns and numbers in the data.
    -   You MUST explicitly state that this is an inference. For example, start your reasoning with "Based on the data showing a low renewables share of X% and high total consumption of Y EJ, a logical inference is that...".

5.  **Format Your Output:** Your final output must be a single, valid JSON object with the following three keys: "answer", "reference", "reasoning".
    -   `"answer"`: Provide a direct and concise answer to the user's question.
    -   `"reference"`: Cite the specific data points from the context that support your answer, weaving them into a readable sentence or paragraph.
    -   `"reasoning"`: Explain the step-by-step logic you used to arrive at the answer. Describe any comparisons or calculations performed.
"""

def process_user_query(user_question: str, data_context: dict) -> dict:
    print(f"\n'{user_question}' 질문 처리 시작...")
    
    try:
        response = client.chat.completions.create(
            model=LLM_MODEL,
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": f"User Question: \"{user_question}\"\n\nJSON Data:\n{json.dumps(data_context)}"}
            ]
        )
        content = response.choices[0].message.content
        parsed_response = json.loads(content)
        print("질문 처리 완료!")
        return parsed_response

    except openai.APIError as e:
        print(f"OpenAI API 오류 발생: {e}")
        return {"error": "API Error", "details": str(e)}
    except json.JSONDecodeError:
        print(f"JSON 파싱 오류 발생. LLM이 유효한 JSON을 반환하지 않았습니다.")
        return {"error": "JSON Decode Error", "raw_content": content}
    except Exception as e:
        print(f"알 수 없는 오류 발생: {e}")
        return {"error": "Unknown Error", "details": str(e)}

user_queries = [
    "What is the Total Energy of Japan in 2050, in the APS scenario?",
    "In 2030 under the Stated Policies Scenario, what is the renewables' share of total energy in the United States?",
    "Based on the 2050 data, which region appears most vulnerable in a global energy transition due to a combination of high total energy consumption and a low share of renewables? Explain the potential risks based on the data."
]

for i, query in enumerate(user_queries):
    result = process_user_query(query, hierarchical_data)
    
    print(f"\n--- 결과 (질문 {i+1}) ---")
    print(json.dumps(result, indent=4, ensure_ascii=False))
    
    if i == 0:
        with open(response_path, "w", encoding="utf-8") as f:
            json.dump(result, f, indent=4, ensure_ascii=False)
        print(f"\n(첫 번째 질문의 답변을 {response_path} 파일에 저장했습니다.)")